<a href="https://colab.research.google.com/github/MostafaELFEEL/Machine-Learning-and-Deep-Learning-Projects/blob/main/SVM_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

# ------------------------------------
# HOG Descriptor Setup
# ------------------------------------
win_size = (32, 32)
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
nbins = 9
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)

# ------------------------------------
# Load CIFAR-100 Dataset
# ------------------------------------
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar100.load_data()
train_labels = train_labels.ravel()
test_labels = test_labels.ravel()

# Dataset Splits
train_size = 10000
val_size = 10000
test_size = 10000

X_train_raw = train_images[:train_size]
y_train = train_labels[:train_size]
X_val_raw = train_images[train_size:train_size + val_size]
y_val = train_labels[train_size:train_size + val_size]
X_test_raw = test_images[:test_size]
y_test = test_labels[:test_size]

# ------------------------------------
# Feature Extraction with HOG
# ------------------------------------
def compute_hog_batch(images):
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        feat = hog.compute(gray)
        features.append(feat.flatten())  # Flatten for classifier
    return np.array(features)

X_train = compute_hog_batch(X_train_raw)
X_val = compute_hog_batch(X_val_raw)
X_test = compute_hog_batch(X_test_raw)

# ------------------------------------
# Feature Scaling
# ------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# ------------------------------------
# Hyperparameter Tuning with Grid Search
# ------------------------------------
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']  # Only used for rbf and poly
}

grid_search = GridSearchCV(svm.SVC(decision_function_shape='ovr'), param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

# ------------------------------------
# Best Model Evaluation
# ------------------------------------
best_model = grid_search.best_estimator_

val_pred = best_model.predict(X_val_scaled)
val_acc = accuracy_score(y_val, val_pred)

test_pred = best_model.predict(X_test_scaled)
test_acc = accuracy_score(y_test, test_pred)

print("Best SVM Parameters:", grid_search.best_params_)
print("Validation Accuracy:", round(val_acc * 100, 2), "%")
print("Test Accuracy:", round(test_acc * 100, 2), "%")

Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best SVM Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Validation Accuracy: 22.74 %
Test Accuracy: 22.82 %
